In [ ]:
import albumentations as albu
import os
from image_segmentation.common import PROJECT_ROOT, DATA_LOADER_SEED
from image_segmentation.data.utils import Augment, load_test_images, load_train_valid_images
from image_segmentation.data.datamodule import DataModule
import tensorflow as tf
from imutils import paths
from typing import Tuple, Sequence
import random

import hydra
from hydra import compose, initialize
from omegaconf import OmegaConf

import cv2
import matplotlib.pyplot as plt

In [ ]:
img_path = os.path.join(PROJECT_ROOT, 'data/membrane', 'train/image/0.png')
mask_path = os.path.join(PROJECT_ROOT, 'data/membrane', 'train/label/0.png')
data_path = [img_path, mask_path]


In [ ]:
img, mask = load_train_valid_images(data_path=data_path)

In [ ]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 18
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(1, 2, figsize=(8, 8))

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(8, 8))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
        
        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original mask', fontsize=fontsize)
        
        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed image', fontsize=fontsize)
        
        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed mask', fontsize=fontsize)

In [ ]:
print(img.shape, mask.shape)

In [ ]:
original_height, original_width = img.shape[:2]

In [ ]:
visualize(img, mask)

In [ ]:
initialize(version_base=None, config_path="../conf", job_name="test_app")
cfg = compose(config_name="default.yaml")

In [ ]:
datamodule = hydra.utils.instantiate(cfg.nn.data, _recursive_=False)
datamodule.prepare_data()

In [ ]:
datamodule.setup()

In [ ]:
train_dataloader = datamodule.train_dataloader()
val_dataloader = datamodule.val_dataloader()
test_dataloader = datamodule.test_dataloader()